# 构建基本聊天机器人

### 安装所需的软件包并配置您的环境

In [1]:
# %%capture --no-stderr
# %pip install -U langgraph langsmith langchain_anthropic

首先创建一个StateGraph。StateGraph对象将我们的聊天机器人的结构定义为“状态机”。我们将添加nodes以表示我们的聊天机器人可以调用的 llm 和函数，并edges指定机器人应如何在这些函数之间转换。

我们的图表现在可以处理两个关键任务：

1、每个都node可以接收当前State作为输入并输出状态更新。

2、messages由于语法add_messages中使用了预建函数，更新将被附加到现有列表而不是覆盖它。Annotated

In [2]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

接下来，添加一个“ chatbot”节点。节点代表工作单元。它们通常是常规的 Python 函数。

**注意**
节点函数如何chatbot将当前State作为输入并返回一个包含messages键“messages”下更新列表的字典。这是所有 LangGraph 节点函数的基本模式。

add_messages我们的函数会将Statellm 的响应消息附加到状态中已有的任何消息中。

In [3]:
# from langchain_anthropic import ChatAnthropic

# llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")


# def chatbot(state: State):
#     return {"messages": [llm.invoke(state["messages"])]}


# # The first argument is the unique node name
# # The second argument is the function or object that will be called whenever
# # the node is used.
# graph_builder.add_node("chatbot", chatbot)

from langchain.llms import Ollama  # 请确保安装了 langchain-ollama 扩展
# from langgraph.builder import GraphBuilder
# from ... import State  # 假设这是你的 State 类定义路径

# 创建 Ollama 模型实例
llm = Ollama(
    base_url="http://10.10.14.3:11434",
    model="deepseek-r1:32b",  # 替换为你的 Ollama 模型名称
)

def chatbot(state: State):
    # 调用 Ollama 模型并返回生成的内容
    return {"messages": [llm.invoke(state["messages"])]}


# 添加 chatbot 节点
graph_builder.add_node("chatbot", chatbot)

/tmp/ipykernel_50925/2058395297.py:20: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


接下来，添加一个entry点。这将告诉我们的图表每次运行时**从哪里开始工作。**

In [4]:
graph_builder.add_edge(START, "chatbot")

同样，设置一个finish点。这将指示图表“任何时候运行此节点，您都可以退出。”

In [5]:
graph_builder.add_edge("chatbot", END)

最后，我们希望能够运行我们的图表。为此，请compile()在图表构建器上调用“”。这将创建一个“ CompiledGraph”，我们可以在状态上使用“调用”。

In [6]:
graph = graph_builder.compile()

get_graph您可以使用方法和其中一种“绘制”方法（例如draw_ascii或）来可视化图形draw_png。draw每个方法都需要额外的依赖项。

In [7]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

现在让我们运行聊天机器人！

**提示：**您可以随时通过输入“quit”、“exit”或“q”退出聊天循环。

In [8]:
# def stream_graph_updates(user_input: str):
#     for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
#         for value in event.values():
#             print("Assistant:", value["messages"][-1].content)

def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        print("Event:", event)  # 打印事件内容
        for value in event.values():
            print("Value:", value)  # 打印值
            if "messages" in value and len(value["messages"]) > 0:
                last_message = value["messages"][-1]
                if hasattr(last_message, "content"):
                    print("Assistant:", last_message.content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Event: {'chatbot': {'messages': ['<think>\n\n</think>\n\n啦啦啦啦啦！好像在唱歌呢，你是在分享一首歌的歌词吗？还是想聊点什么？😊']}}
Value: {'messages': ['<think>\n\n</think>\n\n啦啦啦啦啦！好像在唱歌呢，你是在分享一首歌的歌词吗？还是想聊点什么？😊']}
Goodbye!
